## Joint Graphical Lasso Analysis for MALDI Data
Used for Figure 6

Author: Max Gold

In [1]:
from collections import Counter
import itertools
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib
import networkx as nx
import os
import scanpy as sc
from scipy import stats
import seaborn as sns
import sklearn

import warnings
warnings.filterwarnings('ignore')


In [2]:
## glasso
from gglasso.solver.admm_solver import ADMM_MGL
from gglasso.helper.data_generation import time_varying_power_network, sample_covariance_matrix
from gglasso.helper.experiment_helper import lambda_grid, discovery_rate, error
from gglasso.helper.utils import get_K_identity
from gglasso.helper.experiment_helper import plot_evolution, plot_deviation, surface_plot, single_heatmap_animation
from gglasso.helper.model_selection import aic, ebic

In [ ]:
base_fold = '../../data/'

In [3]:
nfoo = sc.read_h5ad(os.path.join(base_fold, 'maldi_data_vf.h5ad'))

In [4]:
metabs = list(nfoo.var['metabolite'])
mm = dict([[index, row] for index,row in enumerate(metabs)])

In [5]:
## scale data

In [6]:
samps = nfoo.obs['sample'].unique()

cd = {}
for s in samps:
    gg = nfoo[nfoo.obs['sample']==s]
    sc.pp.scale(gg)
    rs = np.cov(np.transpose(gg.X))
    cd[s] = rs

counts = dict(nfoo.obs.groupby("sample").count()['x'])


In [8]:
## categorize samples into others (oal) and LSGN (lal)
oal = ['MDT-AP-0388','MDT-AP-2377', 'MB4143','MB3201']
lal = sorted(set(samps).difference(oal))
nsamps = oal+lal

In [9]:
## organize covriance matrices
S = np.array([cd[x] for x in nsamps])

In [444]:
## setup parameters
K = S.shape[0]
p = S.shape[1]

N = [counts[x] for x in nsamps]
reg = 'GGL'

Omega_0 = get_K_identity(K,p)
Theta_0 = get_K_identity(K,p)
X_0 = np.zeros((K,p,p))

In [445]:
nval = 5
lol1 = list(np.linspace(0.05, 0.25, nval))
lol2 = list(np.linspace(0.01, 0.05, nval))
llo = []
for lambda1 in lol1:
    for lambda2 in lol2:
        sol, info =  ADMM_MGL(S, lambda1, lambda2, reg , Omega_0, Theta_0 = Theta_0, X_0 = X_0, tol = 1e-5, rtol = 1e-5, verbose = False, measure = False)
        Theta_sol = sol['Theta']
        Omega_sol = sol['Omega']
        

        aoo = aic(S, Theta_sol, N)
        boo = ebic(S, Theta_sol, N, gamma = 20)
        llo.append([boo, lambda1, lambda2])
        print(lambda1, lambda2, aoo, boo, np.mean([ (np.count_nonzero(x) - x.shape[1]) / x.shape[1] for x in Theta_sol]))

ADMM terminated after 40 iterations with status: optimal.
0.05 0.01 234127.43510216943 2147508.2463476206 20.735714285714288
ADMM terminated after 39 iterations with status: optimal.
0.05 0.02 257891.02674238198 2238494.828638238 21.464285714285715
ADMM terminated after 39 iterations with status: optimal.
0.05 0.03 282808.127780018 2314494.6786093595 22.017857142857146
ADMM terminated after 39 iterations with status: optimal.
0.05 0.04 304992.0569778451 2355447.281067874 22.221428571428568
ADMM terminated after 40 iterations with status: optimal.
0.05 0.05 329549.0166593343 2403402.5409358116 22.475
ADMM terminated after 35 iterations with status: optimal.
0.1 0.01 474247.133194913 1948300.752479302 15.975
ADMM terminated after 34 iterations with status: optimal.
0.1 0.02 494360.61118467135 2023120.1495359302 16.567857142857143
ADMM terminated after 33 iterations with status: optimal.
0.1 0.03 515210.22924320865 2086149.8805350685 17.025
ADMM terminated after 33 iterations with status:

In [451]:
## optimal params
print(koo[0])
l1opt = koo[0][1]
l2opt = koo[0][2]

[1921069.0000397146, 0.1, 0.01]


In [446]:
Omega_0 = get_K_identity(K,p)
solA, infoA = ADMM_MGL(S, l1opt, l2opt, reg , Omega_0, tol = 1e-8, rtol = 1e-8, verbose = True, measure = True)

------------ADMM Algorithm for Multiple Graphical Lasso----------------
iter	       r_t	       s_t	   eps_pri	  eps_dual
   0	     7.159	     10.44	 0.0001599	 0.0001597
   1	     4.616	     4.069	 0.0001599	 0.0001597
   2	     2.279	     4.349	   0.00016	 0.0001597
   3	     1.193	     3.637	   0.00016	 0.0001597
   4	    0.7845	     2.815	   0.00016	 0.0001597
   5	    0.5625	     2.338	   0.00016	 0.0001597
   6	    0.4189	     2.009	 0.0001601	 0.0001597
   7	    0.3225	     1.758	 0.0001601	 0.0001597
   8	    0.2536	      1.56	 0.0001601	 0.0001597
   9	    0.2026	     1.399	 0.0001601	 0.0001597
  10	    0.1619	     1.266	 0.0001601	 0.0001597
  11	    0.1316	     1.154	 0.0001601	 0.0001597
  12	    0.1079	     1.058	 0.0001601	 0.0001597
  13	   0.09062	    0.9746	 0.0001601	 0.0001597
  14	    0.1112	    0.8451	 0.0001602	 0.0001597
  15	    0.1193	      0.74	 0.0001602	 0.0001597
  16	    0.1188	    0.6549	 0.0001602	 0.0001597
  17	    0.1133	    0.5842	 0.0001602	 0.00015

 165	 0.0001607	 0.0001046	 0.0001603	 0.0001597
 166	 0.0001571	 9.821e-05	 0.0001603	 0.0001597
ADMM terminated after 167 iterations with status: optimal.


In [453]:
## get output from graph

In [454]:
theta = solA['Theta']

In [455]:
## graph functions
def get_graph(a,mm):
    np.fill_diagonal(a, 0)
    G = nx.from_numpy_array(a)
    G = nx.relabel_nodes(G,mm) 
    return G

def get_graph_dict(l, mm):
    return dict([[nsamps[i], get_graph(x, mm)] for i,x in enumerate(l)])

def get_consensus_network(gd, samps, cut):
    c = Counter(list(itertools.chain.from_iterable([gd[x].edges for x in samps])))
    ge = [k for k,v in c.items() if (v >= (len(samps)*cut))]
    gec = dict([[x, c[x]/len(samps)] for x in ge])
    gecl = [gec[x] for x in ge]
    gedf = pd.DataFrame(ge, columns = ['A', 'B'])
    gedf['Percent'] = gecl
    gun = nx.from_pandas_edgelist(gedf, 'A', 'B',['Percent'])
    return gun


In [ ]:
## get metabolite annotations and get indivdual tumor graphs

In [456]:
metabs = list(nfoo.var['metabolite'])
mm = dict([[index, row] for index,row in enumerate(metabs)])

In [457]:
## get networks for each tumor section
ook = get_graph_dict(theta, mm)

In [ ]:
## get consensus networks
anet = get_consensus_network(ook, oal, 0.5)
bnet = get_consensus_network(ook, lal, 0.5)

bb = nx.betweenness_centrality(bnet)
ba = nx.betweenness_centrality(anet)

In [ ]:
## iterate through tumor sections individually to get centrality

In [471]:
meto = mm.values()

In [472]:
bdf = pd.DataFrame(index=meto)
for samp in nsamps:
    uu = ook[samp]
    fo = nx.betweenness_centrality(uu)
    fod = pd.DataFrame(fo, columns = [samp])
    ool = [fo[x] if x in fo.keys() else 0 for x in meto]
    bdf[samp] = ool

In [473]:
bdf = bdf.T
bdf['Subtype'] =  ['SHHa' for _ in range(len(oal))] + ['SHHb' for _ in range(len(lal))]

In [474]:
our = bdf.groupby("Subtype").mean().T
our['diff'] = our['SHHb'] - our['SHHa']

In [475]:
our.sort_values("diff", ascending=False).head(10)

Subtype,SHHa,SHHb,diff
taurine,0.017890,0.029626,0.011736
GMP,0.009467,0.020582,0.011115
Lipid8,0.018271,0.027998,0.009727
C16:1,0.005471,0.014998,0.009527
Phosphoserine,0.013430,0.021786,0.008356
glucose-1-phosphate,0.016427,0.023752,0.007325
Cyclic-ADP-ribose,0.005044,0.011422,0.006378
pantothenate,0.006340,0.012244,0.005904
O-Phosphorylethanolamine,0.010997,0.016348,0.005351
histidine,0.019337,0.024384,0.005047


In [476]:
our.sort_values("diff").head(10)

Subtype,SHHa,SHHb,diff
citrate_isocitrate,0.028278,0.014471,-0.013807
C18:0,0.019296,0.008116,-0.011180
AMP_dGMP,0.028309,0.017699,-0.010611
Uric acid,0.022886,0.012928,-0.009958
uridine,0.019901,0.009991,-0.009909
Lipid2,0.022916,0.013764,-0.009152
inosine,0.023269,0.015621,-0.007648
ribose-phosphate,0.018912,0.011722,-0.007190
guanine,0.020582,0.013480,-0.007102
glucosamine-6-phosphate,0.016204,0.011034,-0.005170


In [ ]:
## compare centrality in consensus networks

In [477]:
cg = list(set(ba.keys()).intersection(bb.keys()))

In [478]:
l = [bb[cg[i]]-ba[cg[i]] for i in range(len(cg))]
ldf = pd.DataFrame(l, index=cg, columns=['diff'])
ldf['B'] = [bb[cg[i]] for i in range(len(cg))]
ldf['A'] = [ba[cg[i]] for i in range(len(cg))]

In [479]:
ldf.sort_values('diff', ascending=False).head(10)

,diff,B,A
histidine,0.019913,0.034285,0.014372
Lipid8,0.019321,0.037139,0.017818
Phosphoserine,0.015980,0.028114,0.012134
deoxyuridine,0.014396,0.030866,0.016469
taurine,0.014176,0.026976,0.012799
O-Phosphorylethanolamine,0.013205,0.020989,0.007785
N-Acetyl-L-alanine,0.012371,0.016256,0.003885
aconitate,0.008550,0.014999,0.006449
glucose-1-phosphate,0.008309,0.024246,0.015937
sn-glycerol-3-phosphate,0.008236,0.021360,0.013124


In [480]:
com = 'taurine'

In [481]:
## edges only in LSGN (SHHb-like) networks
[x for x in bnet.edges - anet.edges if (x[0]==com) or (x[1]==com)]

[('taurine', 'AMP_dGMP'),
 ('taurine', 'GMP'),
 ('taurine', 'UDP'),
 ('taurine', 'cystathionine'),
 ('taurine', 'cyclic-AMP'),
 ('taurine', 'Cyclic-ADP-ribose'),
 ('taurine', 'guanine')]

In [482]:
## edges only in Other (SHHa-like) networks
[x for x in anet.edges - bnet.edges if (x[0]==com) or (x[1]==com)]

[('taurine', 'Nicotinuric acid'),
 ('taurine', 'ribose-phosphate'),
 ('taurine', 'Uric acid'),
 ('taurine', 'histidine')]

In [483]:
## Total edges in LSGN (SHHb-like) networks
[x for x in bnet.edges if (x[0]==com) or (x[1]==com)]

[('taurine', 'N-Acetyl-L-alanine'),
 ('taurine', 'asparagine'),
 ('taurine', 'aspartate'),
 ('taurine', 'O-Phosphorylethanolamine'),
 ('taurine', 'guanine'),
 ('taurine', 'sn-glycerol-3-phosphate'),
 ('taurine', 'Phosphoserine'),
 ('taurine', 'D-gluconate'),
 ('taurine', 'cystathionine'),
 ('taurine', 'deoxyuridine'),
 ('taurine', 'glucosamine-6-phosphate'),
 ('taurine', 'glucose-1-phosphate'),
 ('taurine', 'AMP_dGMP'),
 ('taurine', 'IMP'),
 ('taurine', 'GMP'),
 ('taurine', 'UDP'),
 ('taurine', 'GDP'),
 ('taurine', 'Cyclic-ADP-ribose'),
 ('taurine', 'GlucoseCl'),
 ('taurine', 'cyclic-AMP'),
 ('taurine', 'citrate_isocitrate')]